### Test 2: Machine learning

In this test we will use the entire dataset from the walmart kaggle challenge, do some feature engineering and data munging, then fit a random forest model to our data.

Again, the data is a csv file which contains one line for each scan on their system, with a Upc, Weekday, ScanCount, DepartmentDescription and FinelineNumber.

The VisitNumber column groups our data into baskets - Every unique VisitNumber is a unique basket, with a basket possibly containing multiple scans.

The label is the TripType column, which is Walmarts proprietary way of clustering their visits into categories.

This time we will use the full dataset - we have about 650,000 lines, in about 100,000 baskets. 

If you do need to run this script multiple times, download the dataset from the website rather than redownloading each time, as it's around 30 mb.

Please answer the questions in the cells below them - feel free to answer out of order, but leave comments saying where you carried out the answer. I am working more or less step by step through my answer - Feel free to add on extra predictors if you can think of them.

1\. Import the modules you will use for the rest of the test:

In [1]:
import pandas as pd
import numpy as np

2\. Read in the data, and check its head. The data is available on the website at: http://jeremy.kiwi.nz/pythoncourse/assets/tests/test2data.csv

In [2]:
df = pd.read_csv('test2data.csv')

In [3]:
df.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,999,5,Friday,6.811315e+10,-1,FINANCIAL SERVICES,1000.0
1,30,7,Friday,6.053882e+10,1,SHOES,8931.0
2,30,7,Friday,7.410811e+09,1,PERSONAL CARE,4504.0
3,26,8,Friday,2.238404e+09,2,PAINT AND ACCESSORIES,3565.0
4,26,8,Friday,2.006614e+09,2,PAINT AND ACCESSORIES,1017.0


3\. Fix the Weekday and DepartmentDescription into dummified data. For now they can be seperate dataframes

In [4]:
weekdummies = pd.get_dummies(df.Weekday)
departdummies = pd.get_dummies(df.DepartmentDescription)

4\. Drop the unneeded columns from the raw data - I suggest removing - 'Weekday', 'Upc', 'DepartmentDescription' and 'FinelineNumber' (we could dummify Upc and FineLine, but this will massively increase our data size.)

In [5]:
df = df.drop(['Weekday','Upc','DepartmentDescription','FinelineNumber'],axis=1)

5\. Correct the Dummified data for number bought in each ScanCount. I would recommend something like:

`departdummies.multiply(dat['ScanCount'], axis = 0)`

In [6]:
departdummies = departdummies.multiply(df['ScanCount'], axis = 0)

6\. Concatenate back together the dummy variables with the main dataframe

In [7]:
df1 = pd.concat([df,departdummies,weekdummies],axis=1)
df1 = df1.drop(['ScanCount'],axis=1)

In [8]:
df1.head()

,TripType,VisitNumber,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,...,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,999,5,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,30,7,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,30,7,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,26,8,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,26,8,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


7\. Summarise the data for each basket (hint, if you groupby columns, an .agg() method will not apply to them)

In [9]:
df2 = df1.groupby(['TripType','VisitNumber']).agg('sum')
df2[weekdummies.columns] = np.where(df2[weekdummies.columns]>0,1,0)

8\. Use the reset_index() method to remove your groupings. As we did not cover multiple indices in the lesson, my answer was 

`dat1 = dat1.reset_index()`

In [10]:
df3 = df2.reset_index()

In [11]:
df3.head()

,TripType,VisitNumber,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,...,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,3,106,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,3,121,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,3,153,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,3,162,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,3,164,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


9\. Write a function which takes as an input two lines of your dataframe and returns a distance score between them. Some function of percentage DepartmentDescriptions shared corrected for number of items.

In [12]:
from sklearn.preprocessing import scale
X = df3[weekdummies.columns.append(departdummies.columns).values]
y = df3['TripType'].values
X_scaled = scale(X)

10\. Run this function on a small subset of the dataframe. Use profiling to see the slow points

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.1, random_state=42)

In [14]:
from scipy.spatial.distance import pdist,squareform
dists = pdist(X_test, 'euclidean')
dists1 = squareform(dists)

11\. Compare every line with every line, and store the output (NB, this might take a long time to run). If too long, use a subset, but explicitly state the subset you used.

In [15]:
np.fill_diagonal(dists1, 9999999)

In [16]:
# select the visits with the same X, Weekdays and department purchases, that is, the distance is 0
a,b=np.where(dists1==0)

# to see if those visits with the same X have the same visit types
# calculate the percentage of visits with diffent visit types
visittypes = []
for i in range(len(a)):
    if a[i] < b[i]:
        visittypes.append([a[i],b[i],y_test[a[i]],y_test[b[i]]])
        
visittypes = np.array(visittypes)
(visittypes[:,2] == visittypes[:,3]).mean()

0.7671914451210456

In [17]:
# from the calculation above, about 24% of the visits with the same X have different visit types.
# for example, the two visits below with the same X, but with different type (one is 999 while the other is 3)
print(X_test[9365,:]==X_test[9546,:])
print(y_test[[9365,9546]])

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True]
[999   3]


12\. Which two baskets has the highest similarity? did they share the same TripType?

#### based on the analysis above, we see some baskets are the same in terms of weekday and # of purchases within each department. However, they have different trip type

Thanks for taking the Python Course!

Please save your notebook file as 'your name - test2.ipynb', and email it to jeremycgray+pythoncourse@gmail.com by the 29th of April.